In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
import csv

# Define the perform_random_action function here (already defined earlier)
def perform_random_action(driver):
    actions = [
        "click_on_case_study",
        "click_learn_more",
        "click_on_image",
        "click_on_link",
    ]
    random_action = random.choice(actions)

    if random_action == "click_on_case_study":
        # Randomly select a case study ID and perform the action
        selected_case_study_id = random.choice(case_study_ids)
        case_study_section = driver.find_element(By.ID, selected_case_study_id)
        driver.execute_script("arguments[0].scrollIntoView();", case_study_section)
        driver.execute_script("arguments[0].click();", case_study_section)
    elif random_action == "click_on_image":
        # Implement the code to click on an image (if present)
        images = driver.find_elements(By.TAG_NAME, 'img')
        if images:
            random_image = random.choice(images)
            driver.execute_script("arguments[0].scrollIntoView();", random_image)
            driver.execute_script("arguments[0].click();", random_image)
    elif random_action == "click_on_link":
        # Implement the code to click on a link (if present)
        links = driver.find_elements(By.TAG_NAME, 'a')
        if links:
            random_link = random.choice(links)
            driver.execute_script("arguments[0].scrollIntoView();", random_link)
            driver.execute_script("arguments[0].click();", random_link)
    elif random_action == "click_learn_more":
        # Implement the code to click the "Learn More" button
        learn_more_button = driver.find_element(By.ID, "appx-btn-learnmore-summary-bar-id")
        driver.execute_script("arguments[0].scrollIntoView();", learn_more_button)
        driver.execute_script("arguments[0].click();", learn_more_button)

# Read IP addresses from a CSV file
def read_ip_addresses_from_csv(csv_file):
    ip_addresses = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            ip_addresses.append(row[0])
    return ip_addresses

# List of case study IDs (you can add more as needed)
case_study_ids = [
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:0:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:1:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:2:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:3:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:4:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:5:title",
    "AppxConsultingListingDetail:AppxLayout:listingDetailOverviewTab:appxListingDetailOverviewTabComp:j_id770:j_id771:j_id788:6:title",
]

# Read IP addresses from a CSV file
ip_addresses = read_ip_addresses_from_csv('us_ips.csv')

# List of search keywords
search_keywords = ["techila", "techila global", "techila global services", "techila services"]

for proxy_ip in ip_addresses:
    # Create a Selenium WebDriver object for each proxy IP address and keyword
    print(f'Using proxy IP: {proxy_ip}')

    options = webdriver.FirefoxOptions()
    options.add_argument(f'--proxy-server=http://{proxy_ip}')
    driver = webdriver.Firefox(options=options)

    # URL to navigate to
    url = 'https://appexchange.salesforce.com/'

    try:
        # Randomly select a search keyword
        random_keyword = random.choice(search_keywords)

        # Navigate to the URL
        driver.get(url)

        # Wait for the page to load (you can use WebDriverWait as shown in previous examples)
        time.sleep(5)

        # Execute JavaScript to traverse through the shadow DOM and locate the input element
        script = """
            return document.querySelector("#main > page-container").shadowRoot
                .querySelector("analytics-handler > ace-header").shadowRoot
                .querySelector("div > div > div > header > div.appx-global-header--search > search-type-ahead").shadowRoot
                .querySelector("div > div.input-container > input");
        """

        input_element = driver.execute_script(script)

        if input_element:
            # Clear the input field
            input_element.clear()

            # Send the random keyword to the input field
            input_element.send_keys(random_keyword)

            # Simulate pressing the "Enter" key
            input_element.send_keys(Keys.RETURN)

            print(f'Searching for keyword "{random_keyword}" with proxy IP: {proxy_ip}')

            # Wait for a few seconds for the results page to load
            time.sleep(5)

            # Execute JavaScript to locate and click the "Consultants" button
            script_consultants = """
                return document.querySelector("#main > page-container").shadowRoot
                    .querySelector("analytics-handler > div > x-lazy").shadowRoot
                    .querySelector("div > div.search-title-container > search-switcher").shadowRoot
                    .querySelector("div > ads-button:nth-child(2)").shadowRoot
                    .querySelector("amc-analytics-instrument > button");
            """

            consultants_button = driver.execute_script(script_consultants)

            if consultants_button:
                consultants_button.click()
                print('Consultants button clicked successfully')

                # Wait for a few seconds for the "Consultants" page to load
                time.sleep(5)

                # Execute JavaScript to locate and click the element on the "Consultants" page
                script_element = """
                    return document.querySelector("#main > page-container").shadowRoot
                        .querySelector("analytics-handler > div > x-lazy").shadowRoot
                        .querySelector("div > div.content-container > div.results-container > div.search-container > search-results").shadowRoot
                        .querySelector("div > amc-tiles-grid").shadowRoot
                        .querySelector("div > ads-grid > ads-grid-item > amc-listing-tile").shadowRoot
                        .querySelector("amc-analytics-instrument > div > ads-card > div:nth-child(1) > div:nth-child(2) > ads-content-title > div > a");
                """

                target_element = driver.execute_script(script_element)

                if target_element:
                    target_element.click()
                    print('Target element clicked successfully')

                    # Wait for the new page to load
                    time.sleep(5)

                    # Perform random actions on the new page
                    perform_random_action(driver)

                else:
                    print('Target element not found.')
            else:
                print('Consultants button not found.')

        else:
            print('Input element not found.')

    except Exception as e:
        print("Error:", e)

    # Close the Selenium WebDriver object for the current proxy IP
    
    time.sleep(5)
    driver.quit()


Using proxy IP: 206.232.1.218
Searching for keyword "techila services" with proxy IP: 206.232.1.218
Consultants button clicked successfully
Target element clicked successfully
Using proxy IP: 208.103.166.133
Searching for keyword "techila global services" with proxy IP: 208.103.166.133
Consultants button clicked successfully
Target element clicked successfully
Using proxy IP: 162.244.146.156
Searching for keyword "techila services" with proxy IP: 162.244.146.156
Consultants button clicked successfully
Target element clicked successfully
Using proxy IP: 38.152.32.21
Searching for keyword "techila global services" with proxy IP: 38.152.32.21
Consultants button clicked successfully
Target element clicked successfully
Using proxy IP: 204.217.128.105
Searching for keyword "techila global services" with proxy IP: 204.217.128.105
Consultants button clicked successfully
Target element clicked successfully
Using proxy IP: 193.58.144.26
Searching for keyword "techila services" with proxy IP: 193